# Analyse Exploratoire des Données

## Introduction

L'analyse exploratoire des données (EDA) est une étape fondamentale dans tout projet de data science. Elle permet de :

- **Comprendre** la structure et le contenu des données
- **Identifier** les patterns, anomalies et relations
- **Préparer** les données pour la modélisation
- **Formuler** des hypothèses à tester

Dans ce notebook, nous allons explorer un jeu de données réel en utilisant les bibliothèques Pandas, NumPy, Matplotlib et Seaborn.

---

## 1. Installation et Imports

```python
# Installation des dépendances
# !pip install pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Afficher les graphiques inline
%matplotlib inline

print(f"Versions: pandas={pd.__version__}, numpy={np.__version__}")
```

**Résultat attendu :**
```
Versions: pandas=2.x.x, numpy=1.x.x
```

---

## 2. Chargement des Données

Nous utilisons les données Titanic disponibles dans le répertoire `data/sample/`.

```python
# Chemin vers les données
DATA_PATH = Path("../data/sample/")

# Liste des fichiers disponibles
print("Fichiers disponibles:")
for f in DATA_PATH.glob("*"):
    print(f"  - {f.name}")

# Chargement du dataset
df = pd.read_csv(DATA_PATH / "titanic_sample.csv")

# Informations de base
print(f"\nShape: {df.shape[0]} lignes × {df.shape[1]} colonnes")
```

---

## 3. Exploration de Base

### 3.1 Aperçu des premières lignes

```python
# Afficher les 5 premières lignes
df.head()
```

Cette commande affiche un aperçu des données avec les premières lignes du dataframe.

### 3.2 Informations sur les colonnes

```python
# Types de données et valeurs non-null
df.info()

# Statistiques descriptives
df.describe()
```

### 3.3 Valeurs manquantes

```python
# Compter les valeurs manquantes
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

# Créer un dataframe de résumé
missing_df = pd.DataFrame({
    'Manquantes': missing,
    'Pourcentage': missing_pct.round(2)
})
print(missing_df[missing_df['Manquantes'] > 0])
```

---

## 4. Analyse Univariée

L'analyse univariée consiste à étudier chaque variable séparément.

### 4.1 Variables numériques

```python
# Sélectionner les colonnes numériques
numeric_cols = df.select_dtypes(include=[np.number]).columns
print(f"Colonnes numériques: {list(numeric_cols)}")

# Histogramme
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for col, ax in zip(['Age', 'Fare', 'SibSp', 'Parch'], axes.flat):
    df[col].hist(bins=30, ax=ax, edgecolor='black')
    ax.set_title(f"Distribution de {col}")
    ax.set_xlabel(col)
    ax.set_ylabel('Fréquence')

plt.tight_layout()
plt.savefig('../data/distribution_numerique.png', dpi=150)
plt.show()
```

### 4.2 Variables catégorielles

```python
# Diagramme en barres
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

df['Survived'].value_counts().plot(kind='bar', ax=axes[0], color=['red', 'green'])
axes[0].set_title('Survie (0=Non, 1=Oui)')
axes[0].set_ylabel('Nombre')

df['Pclass'].value_counts().sort_index().plot(kind='bar', ax=axes[1])
axes[1].set_title('Classe du Passager')
axes[1].set_ylabel('Nombre')

df['Sex'].value_counts().plot(kind='bar', ax=axes[2])
axes[2].set_title('Sexe des Passagers')
axes[2].set_ylabel('Nombre')

plt.tight_layout()
plt.savefig('../data/distribution_categorielle.png', dpi=150)
plt.show()
```

---

## 5. Analyse Bivariée

L'analyse bivariée explore les relations entre deux variables.

### 5.1 Corrélations

```python
# Matrice de corrélation
corr_matrix = df.select_dtypes(include=[np.number]).corr()

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
            fmt='.2f', linewidths=0.5)
plt.title('Matrice de Corrélation')
plt.savefig('../data/correlation_heatmap.png', dpi=150)
plt.show()

# Identifier les fortes corrélations
print("\nForte corrélation avec Survived:")
print(corr_matrix['Survived'].sort_values(ascending=False))
```

### 5.2 Relations catégorielles

```python
# Taux de survie par classe
survival_by_class = df.groupby('Pclass')['Survived'].mean() * 100

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Barplot survie par classe
survival_by_class.plot(kind='bar', ax=axes[0], color=['coral', 'steelblue', 'seagreen'])
axes[0].set_title('Taux de Survie par Classe')
axes[0].set_ylabel('Taux de Survie (%)')
axes[0].set_xticklabels(['1ère', '2ème', '3ème'], rotation=0)

# Boxplot âge par classe
df.boxplot(column='Age', by='Pclass', ax=axes[1])
axes[1].set_title('Distribution de l\'Age par Classe')
axes[1].set_xlabel('Classe')
axes[1].set_ylabel('Âge')

plt.suptitle('')
plt.tight_layout()
plt.savefig('../data/analyse_bivariee.png', dpi=150)
plt.show()
```

---

## 6. Visualisations Avancées

### 6.1 Pairplot

```python
# Relations multiples entre variables
sns.pairplot(df[['Age', 'Fare', 'Survived', 'Pclass']], 
            hue='Survived', palette='coolwarm', diag_kind='kde')
plt.savefig('../data/pairplot.png', dpi=150)
plt.show()
```

### 6.2 Violin Plot

```python
# Distribution de l'âge par survie
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x='Survived', y='Age', hue='Sex', 
               split=True, palette='coolwarm')
plt.title('Distribution de l\'Age par Survie et Sexe')
plt.savefig('../data/violin_plot.png', dpi=150)
plt.show()
```

---

## 7. Conclusions et Insights

### Résumé des découvertes

```python
# Synthèse des insights
print("=" * 60)
print("RÉSUMÉ DE L'ANALYSE EXPLORATOIRE")
print("=" * 60)

print(f"\n📊 Dataset: {df.shape[0]} passagers, {df.shape[1]} variables")
print(f"\n🎯 Taux de survie global: {df['Survived'].mean()*100:.1f}%")

print("\n📈 Insights principaux:")
print("  • Les femmes ont un taux de survie plus élevé")
print("  • La classe 1 a un meilleur taux de survie")
print("  • L'âge influence la survie (enfants prioritaires)")
print("  • Le tarif (Fare) corrèle avec la survie")

print("\n⚠️ Points d'attention:")
print("  • Valeurs manquantes dans Age et Cabin")
print("  • Données déséquilibrées (survie vs décès)")

print("\n📋 Recommandations pour la modélisation:")
print("  • Imputer les valeurs manquantes de Age")
print("  • Encoder les variables catégorielles")
print("  • Considérer les interactions (Sex × Class)")
```

---

## 8. Exercices Pratiques

### Exercice 1 : Analyse du housing dataset

```python
# Charger les données housing
housing = pd.read_csv("../data/sample/housing_sample.csv")

# Questions:
# 1. Quelles sont les statistiques de base de la colonne 'price'?
# 2. Quelle est la corrélation entre 'square_feet' et 'price'?
# 3. Créer un scatter plot prix vs superficie
# 4. Analyser la distribution des prix par région
```

### Exercice 2 : Challenge

```python
# Créer une fonction d'analyse rapide
def quick_eda(dataframe):
    """Fonction pour générer un rapport EDA rapide."""
    return {
        'shape': dataframe.shape,
        'numeric_cols': dataframe.select_dtypes(include=[np.number]).columns.tolist(),
        'categorical_cols': dataframe.select_dtypes(include=['object']).columns.tolist(),
        'missing': dataframe.isnull().sum().to_dict()
    }

# Tester avec le dataset
print(quick_eda(df))
```

---

## 9. Ressources

- **Documentation Pandas:** https://pandas.pydata.org/docs/
- **Documentation Seaborn:** https://seaborn.pydata.org/
- **Tutoriel Matplotlib:** https://matplotlib.org/stable/tutorials/index
- **Dataset Titanic:** https://www.kaggle.com/c/titanic

---

*Retour au projet:* [README.md](../README.md)